# Project 2: Making Chatbots!

## Part 1: Rule Based Chatbot (ELIZA)
In this section, you will implement a rule-based Chatbot, ELIZA, using the provided eliza.py file. The eliza.py file contains the rules for the model to follow, you need to complete the code to utilize the file to implement a chat agent while saving the chat history.

In [1]:
import eliza

### Load the Eliza model from eliza.py

In [2]:
eliza = eliza.Eliza()

### Define and open the file to save chat history

In [3]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time to create the file name
file_name = f"ELIZA_CHAT_{now.strftime('%Y_%m_%d_%H_%M_%S')}.txt"
f = open(file_name, "w")

### Define the regex for exit
You need to define an appropriate regex expression that searches for the appropriate words such as 'bye' or 'exit' in the user input to recognize that the user wants to end the chat

In [4]:
import re

def is_end(input_string):
    pattern = re.compile(r'\b(bye|exit|quit)\b', re.IGNORECASE)
    if pattern.search(input_string):
        return True
    return False

### Define a chat loop
In this part, you need to define a loop that asks for user input and prints the user response till the user wants to end the chat

In [5]:
while True:
    user_input = input("USER: ")
    if is_end(user_input):
        # Output the goodbye message, write the last inputs and outputs to the log and close the file
        print("AGENT:  Goodbye!")
        f.write(f"USER: {user_input}\n") # Log the user's last input
        f.write(f"AGENT: Goodbye!\n") # Log the agent's goodbye message
        f.close()
        break
    response = eliza.respond(user_input)
    print("AGENT: ", response)
    f.write(f"USER: {user_input}\n") # Log the user's input
    f.write(f"AGENT: {response}\n") # Log the agent's response

AGENT:  Hi there... how are you today?
AGENT:  How do you feel about being good, how am you?
AGENT:  Would you prefer it if I were not feeling good too?
AGENT:  How long have you been feeling depressed?
AGENT:  since a few years ago.
AGENT:  You seem quite sure.
AGENT:  You seem quite sure.
AGENT:  Please consider whether you can answer your own question.
AGENT:  Very interesting.
AGENT:  I see.  And what does that tell you?
AGENT:  Can you elaborate on that?
AGENT:  How do you feel about being from india, so it makes sense that you love indian cuisine the most?
AGENT:  When do you usually feel good about that?
AGENT:  Why do you say that when im eating?
AGENT:  Goodbye!


Congrats, you are done with part 1. You now need to simply test out your model for 5 chat conversations (minimum 10 utterances in each conversation) and report the results of the human survey.

## Part 2: Corpus Based Chatbot

In this section, you will implement a corpus-based chatbot using the given dialogues.csv corpus. As a part of this task, you will first load the dataset, compute the sentence embeddings for the corpus sentences using the SentenceTransformer Library and then utilize these embeddings for retrieving the most appropriate response.

Note: This part will be slow to run on a CPU based environment (upto 5 minutes), however, it should be very fast on a Colab GPU environment (close to 5 seconds), because of the use of transformer architectures.

In [6]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: sentence_transformers in c:\users\sudha\anaconda3\lib\site-packages (4.0.1)



In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

### Load the dataset
Load the dialogues.csv file using the pandas library.

In [8]:
data = pd.read_csv("dialogues.csv")
data.head()

,emotion,User,Agent
0,sentimental,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,sentimental,This was a best friend. I miss her.,Where has she gone?
2,sentimental,We no longer talk.,Oh was this something that happened because of...
3,sentimental,"Was this a friend you were in love with, or ju...",This was a best friend. I miss her.
4,sentimental,Where has she gone?,We no longer talk.


### Load the SentenceTransformer model
docs: https://sbert.net/docs/sentence_transformer/usage/usage.html

Load the ```all-MiniLM-L6-v2``` sentence transformer model for computing the contextual embeddings.

In [9]:
model = SentenceTransformer('all-MiniLM-L6-v2')

### Compute the sentence embeddings
For the 'User' column of the dataset, compute the sentence embeddings using the sentence transformer model.

In [10]:
user_dialogues = data['User'].tolist()
user_embeddings = model.encode(user_dialogues, convert_to_tensor=True)

### Retrieve the agent response
In the get_response() function, utilize the user_embeddings to retrieve the most similar instance from the data point using cosine similarity. For the selected data point, return the corresponding response in the 'Agent' column of the data as the agent's reponse.

In [11]:
%pip install torch
import torch

Note: you may need to restart the kernel to use updated packages.


In [12]:
def get_response(user_input, data, model, user_embeddings):
    # Convert the input of the user to its sentence embedding
    input_embedding = model.encode(user_input, convert_to_tensor=True)
    
    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(input_embedding, user_embeddings)
    
    # Find the index of the highest cosine similarity using np.argmax.
    best_match_idx = torch.argmax(cosine_scores).item()
    
    # Return the corresponding string for the 'Agent' column
    return data['Agent'].iloc[best_match_idx]


### Define and open the file to save chat history

In [13]:
### Define and open the file to save chat history
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time to create the file name
file_name = f"CORPUS_CHAT_{now.strftime('%Y_%m_%d_%H_%M_%S')}.txt"
f = open(file_name, "w")

### Define a chat loop
In this part, you need to define a loop that asks for user input and prints the user response till the user wants to end the chat. Utilize the same regex expression as before to identify when the user wants to end the chat.

In [14]:
while True:
    user_input = input("USER: ")
    if is_end(user_input):
        # Output the goodbye message, write the last inputs and outputs to the log and close the file
        print("AGENT: Goodbye!")
        f.write(f"USER: {user_input}\n") # Log the user's last input
        f.write("AGENT: Goodbye!\n") # Log the agent's goodbye message
        f.close()
        break
    response = get_response(user_input, data, model, user_embeddings)
    print("AGENT:", response)
    f.write(f"USER: {user_input}\n") # Log the user's input
    f.write(f"AGENT: {response}\n") # Log the agent's response

AGENT: Wow! That's exciting! Was it what you were looking for?
AGENT: Are you going to a restaurant or making it yourself?
AGENT: I have only been looking. Nothing in my price range yet
AGENT: Divorce him
AGENT: Divorce him
AGENT: But I should have known better. I cry about it every day.
AGENT: My grandma died 5 weeks ago. I am still sad.
AGENT: i got payed , went shopping ate dinner, then my friend messaged me asking for his money .... i had totally forgot about this
AGENT: She is the sweetest woman I have ever met
AGENT: And beer.. There has to be beer.
AGENT: Goodbye!


Congrats, you are done with part 2. You now need to simply test out your model for 5 chat conversations (minimum 10 utterances in each conversation) and report the results of the human survey.